In [ ]:
!pip install pyspark

_**Documentacion:** http://spark.apache.org/docs/latest/ml-features.html_

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

### Transformaciones

In [ ]:
df = spark.read.csv(path = "/kaggle/input/pyspark-ml/fake_customers.csv", 
                    inferSchema = True, header = True)

df.show()

### StringIndexer

Para transformar los valores categoricos a numericos.

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer1 = StringIndexer(inputCol = "Group",
                         outputCol = "GroupIndex")

indexed1 = indexer1.fit(df).transform(df)

indexed1.show()

In [ ]:
indexer2 = StringIndexer(inputCol = "Name",
                         outputCol = "NameIndex")

indexed2 = indexer2.fit(indexed1).transform(indexed1)

indexed2.show()

### VectorIndexer
Para transformar los datos a un vector denso. Esto es obligatorio para los modelos de ML en PySpark.

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ["Phone", "GroupIndex", "NameIndex"],
                            outputCol = "features")

output = assembler.transform(indexed2)

output.select(["Phone", "GroupIndex", "NameIndex", "features"]).show()

## LinearRegression

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
data = spark.read.format("libsvm").load("/kaggle/input/pyspark-ml/sample_linear_regression_data.txt")

data.show()

Este es el formato que Spark usa para hacer ML, dos columnas, una llamada "label" y otra de "features", que serían "y" y "X" respectivamente.

La columna "label" o "y" debe ser una columna numerica, al igual que cuando haciamos modelos de SciKit-Learn.

La columna "features" o "X" esta formada por vectores , estos vectores son los elementos agrupados de las columnas iniciales.

In [ ]:
# Inicializamos el modelo
lr = LinearRegression(featuresCol = "features",
                      labelCol = "label",
                      predictionCol = "prediction")

# Nota: si en el DataFrame las columnas tienen otros nombres podemos decirle al modelo que trabaje con esos nombres
# Por convencion "X" se llama "features", "y" se llama "label" y las predicciones se llaman "prediction"
# Y esos son los valores por defecto de los modelo de ML.

# Entrenamos el modelo 
model = lr.fit(data)

In [ ]:
# Podemos imprimir los coeficientes de la regresion
print("Coeficientes: {}".format(str(model.coefficients)))
print("\n")
print("Intercepcion:{}".format(str(model.intercept)))

### Summary

In [ ]:
# El modelo tiene el metodo .summary
# Este summary se hace sobre el set de entrenamiento y calcula las metricas del modelo
summary = model.summary

In [ ]:
summary.residuals.show()
# .residuals es la diferencia entre el valor real y el valor predicho 

print("RMSE: {}".format(summary.rootMeanSquaredError))
print("r2: {}".format(summary.r2))

### Train/Test Split

No existe la función **train_test_split**... Pero los objetos DataFrames de Spark tienen un método que hace lo mismo que **train_test_split**.

In [ ]:
data.show(3)

In [ ]:
# .randomSplit()

train, test = data.randomSplit(weights = [0.7, 0.3], seed = 42)

In [ ]:
train.show(3)

test.show(3)

In [ ]:
model = lr.fit(train)

In [ ]:
y_hat = model.evaluate(test)

In [ ]:
y_hat.predictions.show()

In [ ]:
y_hat.residuals.show()
print("RMSE: {}".format(y_hat.rootMeanSquaredError))

Hasta aquí llegaría el modelo, ahora vamos a ver como hacer predicciones de datos nuevos.

Vamos a trabajar con la columna de "features" de test, así solo nos quedamos con "X" y no sabemos el valor de "y".

In [ ]:
nueva_data = test.select("features")
nueva_data.show(3)

In [ ]:
# En lugar de usar .evaluate() vamos a usar .transform() y esto nos retorna un nuevo DataFrame
y_hat = model.transform(nueva_data)
y_hat.show()

### Guardar el modelo

In [ ]:
model.save(path = "modelo.model")

In [ ]:
################################################################################################################################